In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
from text_loader import TextLoader 

In [4]:
num_layers  = 3
hidden_size = 512
batch_size  = 1
max_length  = 1000

loader = TextLoader("data/hamlet.txt")
vocab_size = len(loader.vocab)

In [5]:
X = tf.placeholder(tf.int32, [None, 1])
x_one_hot = tf.one_hot(X, vocab_size)

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, states = tf.nn.dynamic_rnn(cells, x_one_hot, 
                                    initial_state=initial_state, dtype=tf.float32)

outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.fully_connected(outputs, vocab_size,
                                activation_fn=None)
y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

In [25]:
sentence = list()
# 시작 글자 생성
sentence += loader.X[:10].tolist()
print("Start with:", "".join([loader.words[char] for char in sentence]))

saver = tf.train.Saver()
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "checkpoints/char-rnn_10000")
    
    # 매 이터레이션마다 글자 하나씩 생성
    pred_char, state = sess.run([pred, states], feed_dict={X:[sentence[:]]})
    for i in range(1000):
        pred_char, state = sess.run([pred, states], 
            feed_dict={X:[[sentence[-1]]], initial_state: state}) # 이전 스텝에 갖고 있는 state 값을 다음 스텝에 넣어줌
        sentence.append(pred_char[0][-1])
        
sentence = [loader.words[char] for char in sentence]
print("".join(sentence))

Start with: <START>	HAMLET


INFO:tensorflow:Restoring parameters from checkpoints/char-rnn_10000
<START>	HAMLET

ACT II



SCENE I	A room in the castle.


	[Enter KING CLAUDIUS, QUEEN GERTRUDE, POLONIUS,
	OPHELIA, ROSENCRANTZ, and GUILDENSTERN]

	How now, my lord! I will the king hear this piece of work?

LORD POLONIUS	And the queen too, and that presently.

HAMLET	Bid the players make haste.

	[Exit POLONIUS]

	He tells me, my dear Gertrude, he hath founds: I have been is twice a child.

HAMLET	I will prophesy here,
	You any the compuced and the chief head
	Of this his nephew's purpose,--to suppress
	His further gait herein; in that the levies,
	The lists and full proportions, are all made
	Out of his subject: and we here dispatch
	You, good Cornelius, and you, Voltimand,
	For bearers of this greeting to old Norway;
	Giving to you no further personal power
	To business with the king, more than the scope
	Of these delated articles allow.
	Farewell, and let your haste commend your duty